In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('kdd').getOrCreate()

In [8]:
df = spark.read.csv('kddcup.data_10_percent.gz',
 inferSchema=True, header=False)

In [9]:
df.count()

494021

In [10]:
from pyspark.sql.functions import col

In [11]:
str(df.columns)

"['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40', '_c41']"

In [12]:
cols = ['_c0', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', 
 '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', 
 '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', 
 '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', 
 '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', 
 '_c39', '_c40']
for col_name in cols:
 df = df.withColumn(col_name, col(col_name).cast('float'))

In [13]:
df.show(3)

+---+---+----+---+-----+------+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|_c0|_c1| _c2|_c3|  _c4|   _c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|   _c41|
+---+---+----+---+-----+------+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-------+
|0.0|tcp|http| SF|181.0|5450.0|0.0|0.0|0.0|0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 8.0| 8.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 9.0| 9.0| 1.0| 0.0|0.11| 0.0| 0.0| 0.0| 0.0| 0.0|normal.|
|0.0|tcp|http| SF|239.0| 486.0|0.0|0.0|0.0|0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 8.0| 8.0| 0.0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0|19.0

In [14]:
df.groupBy('_c41').count().show(30)

+----------------+------+
|            _c41| count|
+----------------+------+
|    warezmaster.|    20|
|          smurf.|280790|
|            pod.|   264|
|           imap.|    12|
|           nmap.|   231|
|   guess_passwd.|    53|
|        ipsweep.|  1247|
|      portsweep.|  1040|
|          satan.|  1589|
|           land.|    21|
|     loadmodule.|     9|
|      ftp_write.|     8|
|buffer_overflow.|    30|
|        rootkit.|    10|
|    warezclient.|  1020|
|       teardrop.|   979|
|           perl.|     3|
|            phf.|     4|
|       multihop.|     7|
|        neptune.|107201|
|           back.|  2203|
|            spy.|     2|
|         normal.| 97278|
+----------------+------+



In [15]:
str(df.columns)

"['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40', '_c41']"

In [16]:
train_data,test_data = df.randomSplit([0.8,0.2])

In [17]:
test_data .groupBy('_c41').count().show(30)

+----------------+-----+
|            _c41|count|
+----------------+-----+
|    warezmaster.|    4|
|          smurf.|56069|
|            pod.|   58|
|           nmap.|   43|
|           imap.|    2|
|   guess_passwd.|    7|
|        ipsweep.|  244|
|      portsweep.|  242|
|          satan.|  327|
|           land.|    9|
|     loadmodule.|    3|
|      ftp_write.|    3|
|buffer_overflow.|    8|
|    warezclient.|  196|
|       teardrop.|  192|
|        neptune.|21496|
|           back.|  421|
|            spy.|    1|
|         normal.|19626|
+----------------+-----+



In [26]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [20]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [21]:
# Import class for creating a pipeline
from pyspark.ml import Pipeline

In [23]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier

In [27]:
# Convert categorical strings to index values
indexer1 = StringIndexer(inputCol='_c1', outputCol='c1_idx')
indexer2 = StringIndexer(inputCol='_c2', outputCol='c2_idx')
indexer3 = StringIndexer(inputCol='_c3', outputCol='c3_idx')
indexer41 = StringIndexer(inputCol='_c41', outputCol='c41_idx')
# One-hot encode index values
onehot = OneHotEncoder(
 inputCols=['c1_idx', 'c2_idx', 'c3_idx'],
 outputCols=['c1_dummy', 'c2_dummy', 'c3_dummy']
)
# Assemble predictors into a single column
assembler = VectorAssembler(inputCols=['_c0', 'c1_dummy', 
 'c2_dummy', 'c3_dummy', 
 '_c4', '_c5', '_c6', 
 '_c7', '_c8', '_c9', 
 '_c10', '_c11', '_c12', 
 '_c13', '_c14', 
 '_c15', '_c16', '_c17', 
 '_c18', '_c19', 
 '_c20', '_c21', '_c22', 
 '_c23', '_c24', 
 '_c25', '_c26', '_c27', 
 '_c28', '_c29', 
 '_c30', '_c31', '_c32', 
 '_c33', '_c34', 
 '_c35', '_c36', '_c37', 
 '_c38', '_c39', '_c40'], 
 outputCol='features')
# A linear regression object
dtc = DecisionTreeClassifier(featuresCol='features', 
 labelCol='c41_idx', 
 predictionCol='prediction')

In [28]:
# Construct a pipeline
pipeline = Pipeline(stages=[indexer1, indexer2, indexer3, 
 indexer41, onehot, assembler, dtc])

In [29]:
# Train the pipeline on the training data
pipeline = pipeline.fit(train_data)

In [30]:
# Make predictions on the testing data
predictions = pipeline.transform(test_data)

In [31]:
# Inspect results
predictions.select("prediction", "c41_idx").show(5)

+----------+-------+
|prediction|c41_idx|
+----------+-------+
|       2.0|   10.0|
|       2.0|   10.0|
|       5.0|    5.0|
|       5.0|    5.0|
|       5.0|    5.0|
+----------+-------+
only showing top 5 rows



In [32]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [33]:
# Select (prediction, true label) and compute test error
acc_evaluator = MulticlassClassificationEvaluator(labelCol="c41_idx", 
 predictionCol="prediction", 
 metricName="accuracy")

In [34]:
#important: need to cast to float type, and order by prediction, else it won't wo
preds_and_labels = predictions.select(['prediction','c41_idx'])\
 .withColumn('c41_idx', col('c41_idx')\
 .cast("float")).orderBy('prediction')
#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','c41_idx'])


In [35]:
acc_evaluator.evaluate(preds_and_labels)

0.9937140604945882

In [36]:
from pyspark.mllib.evaluation import MulticlassMetrics

In [37]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [38]:
# Confusion matrix 
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

C:\spark\spark-3.5.0-bin-hadoop3\python\pyspark\sql\context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [39]:
print(metrics.confusionMatrix().toArray())

[[5.6069e+04 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00]
 [0.0000e+00 2.1469e+04 2.6000e+01 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00]
 [0.0000e+00 0.0000e+00 1.9623e+04 1.0000e+00 1.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  1.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00]
 [0.0000e+00 0.0000e+00 1.5000e+01 4.0600e+02 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00]
 [0.0000e+00 0.0000e+00 2.9000e+01 0.0000e+00 2.9800e+02 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0

In [40]:
import pandas as pd
pd.set_option('display.max_columns', 30)

In [41]:
matrix = pd.DataFrame(metrics.confusionMatrix().toArray())

In [42]:
matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,56069.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,21469.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,19623.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,15.0,406.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,29.0,0.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,33.0,0.0,0.0,211.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,180.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,192.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Make new prediction


In [43]:
# New data
df_new = spark.read.csv('kddcup.testdata.unlabeled_10_percent.gz',
 inferSchema=True,
 header=False)

In [44]:
df_new.count()

311029

In [45]:
str(df_new.columns)

"['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40']"

In [46]:
# Make predictions on the testing data
predictions_new = pipeline.transform(df_new)

In [47]:
predictions_new.select('features', 'prediction').show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[8,68,78,96,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[2,5,68,78,7...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[2,5,68,78,7...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[0,2,6,68,78...|       2.0|
|(115,[2,5,68,78,7...|       2.0|
|(115,[2,5,68,78,7...|       2.0|
|(115,[2,5,68,78,7...|       2.0|
|(115,[4,68,78,79,...|       2.0|
|(115,[4,68,78,79,...|       2.0|
+--------------------+----------+
only showing top 20 rows

